Copyright 2022 Huawei TECHNOLOGIES CO.LTD
Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License"); { display-mode: "form" }
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 基于布朗运动模型的欧式期权蒙特卡洛求解(MonteCarloSimulationPricing)

In [15]:
pip install mindspore-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 337.4 MB 16 kB/s 
     |████████████████████████████████| 281 kB 58.2 MB/s 
  Attempting uninstall: psutil
    Found existing installation: psutil 5.4.8
    Uninstalling psutil-5.4.8:
      Successfully uninstalled psutil-5.4.8


In [ ]:
# !pip install --upgrade mindspore
# !pip install mind-quant-finance
# @title temporary install
!pip install -e. --upgrade

如果您的环境并未本地安装此第三方库，请确保您的运行目录在mind-quant-finance上，对此您可以本地调用

In [12]:
#import matplotlib.pyplot as plt #可用于数据可视化
from concurrent.futures import process
import argparse
import time
import mindspore.context as context
from mind_quant_finance.engine.mc.MonteCarloSimulationPricing import MonteCarloSimulationPricing

[WARNING] ME(686459:139995613804352,MainProcess):2022-10-16-14:13:51.469.536 [mindspore/run_check/_check_version.py:140] MindSpore version 1.7.0 and cuda version 11.2.72 does not match, please refer to the installation guide for version matching information: https://www.mindspore.cn/install


## 基于布朗运动模型的欧式期权蒙特卡洛模拟方法
此函数可以返回两个或三个张量（MindSpore格式的矩阵）、“期权价格”、“期权价值”以及一个“期权价格变化过程”矩阵（可选）
### 公式如下：
由``d_St=（r-q）* St* dt+σ* St* dzt ``

可得 ``△St=St* （r-q）* △t+σ* St* ε* √(△t)``

其中：

"△St"为期权在t时刻的价格与在t-1时刻的价格之差，
"St"为期权在t时刻的价格，
"r"为连续复合无风险利率，
"q"为期权红利率，若期权无红利，则q=0，
"△t"为单位时间步长，一般设置为1，
"σ"为期权波动率，一般由该期权历史变动幅度所决定，也可能为估计得出，

此外，为了计算一些受价格限制的证券产品的价格变化率，取上面公式两边的对数定义算子


## 基于Mind-Quant-Finance的MonteCarloSimulationPricing方法模拟计算期权价格

### 求解器参数
您可以使用``mce=MonteCarloSimulationPricing(dim=1,batchsize=1,num_paths=1000,Process=False)``定义求解器

其中：
dim（int）：维度；期权资产数量。
batchsize（int）：模拟环境的个数。每个模拟的宏观环境都是恒定的（您可以理解相同的宏观经济环境：增长、衰退、萧条和繁荣）。当前未定义不同宏观环境，当batchsize=n时，仅相当于模拟了每个期权模拟了n*num_paths次
num_paths（int）：每个batchsize大小中的模拟数。
Process（Bool）：是否输出价格变化过程，可用于数据导出和可视化。
支持的平台：
``GPU``
### 输入参数求解
你可以使用以下示例``out=mce.construct(pv=[30],sigma=[0.2],q=[0.01],rf=[0.05],num_timesteps=365,dt=1)``进行期权价格求解

其中：
pv：各个期权当前价值，现值；大小为1*dim。
sigma：各个期权预估波动率；大小为1*dim。
q：各个期权红利率；大小为1*dim。
rf：各个期权的无风险收益率；同一国家或地区无风险收益率一般相同，大小为1*dim。
num_timesteps:模拟时间次数或长度。
dt：单位时间步长，默认为1，即1天。
## 求解示例
示例一：
求解某无红利现值为30、波动率为20%的欧式期权，在市场无风险利率为5%，以单位时间1为时间步长模拟365天，该期权的模拟价值和价格；对此，您希望模拟出1个大环境的1000次结果

In [23]:
context.set_context(
        mode=context.GRAPH_MODE,
        device_target="GPU",
        device_id=0,
        save_graphs=False
)

In [24]:
mce=MonteCarloSimulationPricing(dim=1,batchsize=1,num_paths=1000)
out=mce.construct(pv=[30],sigma=[0.2],q=[0.],rf=[0.05],num_timesteps=365,dt=1)
print(out)

[WARNING] KERNEL(686459,7f5344d4b740,python):2022-10-16-14:26:10.383.082 [mindspore/ccsrc/plugin/device/gpu/kernel/gpu_kernel_factory.cc:93] ReducePrecision] Kernel [StandardNormal] does not support int64, cast input 0 to int32.
[WARNING] PRE_ACT(686459,7f5344d4b740,python):2022-10-16-14:26:10.383.638 [mindspore/ccsrc/plugin/device/gpu/optimizer/reduce_precision_fusion.cc:84] Run] Reduce precision for [StandardNormal] input 0


(Tensor(shape=[1, 1000, 1], dtype=Float32, value=
[[[ 3.04653076e+02],
  [ 4.29601990e+02],
  [ 2.81714844e+02],
  ...
  [ 3.32874207e+02],
  [ 3.17451569e+02],
  [ 4.10531799e+02]]]), Tensor(shape=[1, 1000, 1], dtype=Float32, value=
[[[ 2.74653076e+02],
  [ 3.99601990e+02],
  [ 2.51714844e+02],
  ...
  [ 3.02874207e+02],
  [ 2.87451569e+02],
  [ 3.80531799e+02]]]))


示例二：

求解某有红利现值为30、波动率为20%，红利率为1%的欧式期权，在市场无风险利率为5%，以单位时间1为时间步长模拟365天，该期权的模拟价值和价格:对此，您希望模拟出1个大环境的1000次结果，并希望得到模拟过程的价值

In [25]:
mce=MonteCarloSimulationPricing(dim=1,batchsize=1,num_paths=1000,Process=True)
out=mce.construct(pv=[30],sigma=[0.2],q=[0.01],rf=[0.05],num_timesteps=365,dt=1)
print(out)

[WARNING] KERNEL(686459,7f5344d4b740,python):2022-10-16-14:26:13.447.270 [mindspore/ccsrc/plugin/device/gpu/kernel/gpu_kernel_factory.cc:93] ReducePrecision] Kernel [StandardNormal] does not support int64, cast input 0 to int32.
[WARNING] PRE_ACT(686459,7f5344d4b740,python):2022-10-16-14:26:13.447.431 [mindspore/ccsrc/plugin/device/gpu/optimizer/reduce_precision_fusion.cc:84] Run] Reduce precision for [StandardNormal] input 0


(Tensor(shape=[1, 1000, 1], dtype=Float32, value=
[[[ 2.77308929e+02],
  [ 4.46287079e+02],
  [ 2.43178848e+02],
  ...
  [ 4.24724792e+02],
  [ 1.17911682e+02],
  [-9.23774796e+01]]]), Tensor(shape=[1, 1000, 1], dtype=Float32, value=
[[[ 2.47308929e+02],
  [ 4.16287079e+02],
  [ 2.13178848e+02],
  ...
  [ 3.94724792e+02],
  [ 8.79116821e+01],
  [-1.22377480e+02]]]), Tensor(shape=[1, 1000, 365, 1], dtype=Float32, value=
[[[[ 2.51600914e+01],
   [ 2.37380123e+01],
   [ 3.19897518e+01],
   ...
   [ 2.60247879e+01],
   [ 3.38967819e+01],
   [ 3.05550537e+01]],
  [[ 2.97146549e+01],
   [ 2.67939625e+01],
   [ 3.41790924e+01],
   ...
   [ 3.40802578e+04],
   [ 2.47809492e+04],
   [ 2.36146660e+04]],
  [[ 3.17810936e+01],
   [ 3.31469154e+01],
   [ 3.22763901e+01],
   ...
   [ 9.92917633e+00],
   [ 8.37481308e+00],
   [ 8.34232521e+00]],
  ...
  [[ 1.70924129e+01],
   [ 2.03536491e+01],
   [ 2.16285191e+01],
   ...
   [ 6.82703955e+03],
   [ 8.06387207e+03],
   [ 9.22520020e+03]],
  [[ 2.9227

示例三：

求解某有红利现值为30、波动率为20%，红利率为1%的欧式期权A，和无红利现值为20、波动率为10%的欧式期权B，在涨跌幅限制为10%的环境中，第一天市场无风险利率为5%，第二天市场无风险利率为4%，以单位时间1为时间步长模拟2天，该期权的模拟价值和价格:对此，您希望模拟出1个大环境的1000次结果，并希望得到模拟过程的价值

In [26]:
mce=MonteCarloSimulationPricing(dim=2,batchsize=1,num_paths=1000,Process=True)
out=mce.construct(pv=[30,20],sigma=[0.2,0.1],q=[0.01,0.],rf=[0.05,0.04],num_timesteps=2,dt=1,lr=0.1)
print(out)

(Tensor(shape=[1, 1000, 2], dtype=Float32, value=
[[[ 3.09601231e+01,  2.01757984e+01],
  [ 2.78259506e+01,  2.40000000e+01],
  [ 2.86052151e+01,  2.34910393e+01],
  ...
  [ 3.60000000e+01,  2.40000000e+01],
  [ 3.50224876e+01,  1.86110210e+01],
  [ 3.55174561e+01,  2.05344372e+01]]]), Tensor(shape=[1, 1000, 2], dtype=Float32, value=
[[[ 9.60123062e-01,  1.75798416e-01],
  [-2.17404938e+00,  4.00000000e+00],
  [-1.39478493e+00,  3.49103928e+00],
  ...
  [ 6.00000000e+00,  4.00000000e+00],
  [ 5.02248764e+00, -1.38897896e+00],
  [ 5.51745605e+00,  5.34437180e-01]]]), Tensor(shape=[1, 1000, 2, 2], dtype=Float32, value=
[[[[ 2.79601250e+01,  1.99281883e+01],
   [ 3.07561378e+01,  2.01749096e+01]],
  [[ 3.01631680e+01,  2.20000000e+01],
   [ 2.78132401e+01,  2.42000008e+01]],
  [[ 2.96654358e+01,  2.14910412e+01],
   [ 2.86170387e+01,  2.36401463e+01]],
  ...
  [[ 3.30000000e+01,  2.20000000e+01],
   [ 3.62999992e+01,  2.42000008e+01]],
  [[ 3.20224838e+01,  1.91290245e+01],
   [ 3.5224731

[WARNING] KERNEL(686459,7f5344d4b740,python):2022-10-16-14:26:24.847.908 [mindspore/ccsrc/plugin/device/gpu/kernel/gpu_kernel_factory.cc:93] ReducePrecision] Kernel [StandardNormal] does not support int64, cast input 0 to int32.
[WARNING] PRE_ACT(686459,7f5344d4b740,python):2022-10-16-14:26:24.848.058 [mindspore/ccsrc/plugin/device/gpu/optimizer/reduce_precision_fusion.cc:84] Run] Reduce precision for [StandardNormal] input 0
